# Plotting Cyclotomic Rings

In this notebook, let us explore the structure and symmetries of cyclotomic rings.

In [ ]:
:dep tilezz = { path = ".." }
:dep plotters = { version = "*" }

// import everything we will need
use std::collections::HashSet;

use tilezz::cyclotomic::*;
use tilezz::cyclotomic::geometry::point_mod_rect;

use plotters::prelude::*;
use plotters::coord::Shift;
use plotters::evcxr::evcxr_bitmap_figure;
use tilezz::plotters::{plot_points, rainbow};
use tilezz::plotutils::{tile_bounds, P64, R64};

## Visualizing Points Reachable In A Fixed Unit Square

The naive approach to compute points in the ring would be doing some simple breadth-first search from the origin, but this is inconvenient if all we want in the end are points within one unit square.

But all starting points are equal. So we can limit our attention to a single unit square and all other neighborhoods will be the same (modulo translation).

So let us explore the points of the ring that we can reach inside the unit square with corners $(0,0)$ and $(1,1)$, i.e. we explore reachable ring elements starting from $0, 1, i$ and $1+i$.

To saturate our set of points, we will use the following algorithm:

* Given a number of rounds $n \in \mathbb{N}$, in each round $i$ go in all possible directions from every point that was reached in round $i-1$ the first time.
* If the point is outside the unit square, normalize it back (by subtracting or adding $1$ or $i$ as often as needed).

Note that the normalized points would have been reached from some adjacent unit square, i.e. some other Gaussian integer, by symmetry.

This way we can populate points inside a single unit square without thinking about the true predecessor points (that can be arbitrarily far away).

The resulting set of points in the unit square is guaranteed to be reachable from *some* $z \in \mathbb{Z}[\zeta_4]$ in at most $n$ steps.

As by assumption we only consider rings that include $\mathbb{Z}[\zeta_4]$ as a subring, all resulting points are valid.

Here is the code for the actual algorithm, the code is more generic so we can do some other experiments later:

In [ ]:
/// Compute the levels of points reachable within the unit square from any Gaussian integer in n steps.
fn explore<ZZ: ZZType + HasZZ4>(n: usize, mod_unit_square: bool) -> Vec<Vec<ZZ>>
where
    <ZZ as IsComplex>::Field: From<(<ZZ as IsRingOrField>::Real, <ZZ as IsRingOrField>::Real)>,
    {
    // we start at the corners of the unit square
    let start_pts: &[ZZ] = if mod_unit_square { &[ZZ::zero(), ZZ::one(), ZZ::one_i(), ZZ::one() + ZZ::one_i()] } else {&[ZZ::zero()]};

    let mut visited: HashSet<ZZ> = HashSet::from_iter(start_pts.to_vec().into_iter());
    let mut round_pts: Vec<Vec<ZZ>> = Vec::new();
    round_pts.push(start_pts.to_vec());

    let unit_square: (ZZ, ZZ) = ((0, 0).into(), (1, 1).into());
    // in each round, we go one unit step in every possible direction
    for i in 1..=n {
        let last = round_pts.last().unwrap();
        let mut curr: Vec<ZZ> = Vec::new();
        for p in last.iter() {
            for i in 0..ZZ::turn() {
                let mut p_dest: ZZ = *p + ZZ::unit(i);
                if mod_unit_square {
                    // normalize back into unit square (if enabled)
                    p_dest = point_mod_rect(&p_dest, &unit_square).coerce_ring();
                }
                let p_dest = p_dest;
                
                if !visited.contains(&p_dest) {
                    visited.insert(p_dest);
                    curr.push(p_dest);
                }
            }
        }
        print!("{}{}", curr.len(), if i==n { "" } else {"+"}); // print number of new points
        if i % 10 == 0 {
            println!("")
        }
        round_pts.push(curr);
    }
    let num_pts: usize = round_pts.iter().map(|v| v.len()).sum();
    println!("\n= {num_pts}");
    return round_pts;
}

And here is some utility code for plotting:

In [ ]:
/// Helper function to plot the points with given settings into a drawing area.
pub fn render<'a, DB: DrawingBackend>(
    da: &DrawingArea<DB, Shift>,
    ((x_min, y_min), (x_max, y_max)): R64,
    point_levels: &[Vec<P64>],
    level_styles: &[(i32, ShapeStyle)],
    offset: usize,
    stride: u32,
) {
    // prepare coordinate system
    let mut chart = ChartBuilder::on(&da).x_label_area_size(20).y_label_area_size(40)
        .build_cartesian_2d(x_min..x_max, y_min..y_max).unwrap();
    chart.configure_mesh().draw().unwrap();
    // plot points level by level with the correct style
    for i in (0..point_levels.len()).step_by(stride as usize).rev() {
        let (sz, st) = level_styles[i as usize];
        plot_points(&mut chart, point_levels[i as usize].as_slice(), |_| format!("{}", i+offset), sz, st);
    }
}

/// Generate combinations of point sizes and colors
fn get_styles(n: usize, sz: Option<i32>) -> Vec<(i32, ShapeStyle)> {
    // get a rainbow color gradient
    let colors = rainbow(n as u32 + 1, 1.);
    (0..=n).collect::<Vec<_>>().iter().map(|i| (
        match sz { Some(size) => size, None => (40. * (0.99_f64.powi(*i as i32))) as i32, },
        colors[*i as usize].filled().into(),
    )).collect()
}

fn prepare_render<ZZ: ZZType + HasZZ4>(
    num_rounds: usize,
    mod_unit_square: bool,
    pt_size: Option<i32>,
) -> (Vec<Vec<P64>>, R64, Vec<(i32, ShapeStyle)>)
where
    <ZZ as IsComplex>::Field: From<(<ZZ as IsRingOrField>::Real, <ZZ as IsRingOrField>::Real)>,
{
    let points: Vec<Vec<P64>> = explore::<ZZ>(num_rounds, mod_unit_square)
        .iter()
        .map(|v| v.iter().map(|p| p.xy()).collect())
        .collect();

    let bounds = tile_bounds(
        points
            .iter()
            .map(|p| <(P64, P64) as Into<[P64; 2]>>::into(tile_bounds(p.iter())).into_iter())
            .flatten()
            .collect::<Vec<P64>>()
            .as_slice(),
    );
    let styles = get_styles(points.len(), pt_size);

    (points, bounds, styles)
}

First, let us compute the points in $\mathbb{Z}[\zeta_{12}]$ for some fixed number of rounds:

In [ ]:
let (points, bounds, styles) = prepare_render::<ZZ12>(64, true, None);

Already here some interesting phenomena are showing up in $\mathbb{Z}[\zeta_{12}]$. Let $i$ be the round and $n_i$ the number of points we discover. Then:

* if $i$ is even, then in round $i+1$ there are also $n_i$ new points
* if in round $i$ we discovered $n_i$ points, in round $i+2$ there will be $n_i + 8$ new points

This number sequence looks like it has a polynomial closed form. The pattern of every next two rounds having the same number of new points is, I guess, due to certain linear dependencies between the directional unit steps. Probably all this can be explained by some well-known mathematical properties.

Now let us plot the points. It's a lot of points, so let us look at different subsets and skip a bunch of exploration steps, to get a clearer picture: *(I recommend using the notebook in full-width mode)*

In [ ]:
evcxr_figure((4000,4000), |root| {
    let areas: Vec<_> = root.split_evenly((2, 2)).into_iter().map(|area| area.margin(40, 40, 40, 40)).collect();

    render(&areas[0], bounds, points.as_slice(), styles.as_slice(), 0, 4);
    render(&areas[1], bounds, points.as_slice(), styles.as_slice(), 0, 5);
    render(&areas[2], bounds, points.as_slice(), styles.as_slice(), 0, 7);
    render(&areas[3], bounds, points.as_slice(), styles.as_slice(), 0, 8);  

    Ok(())
})

In [ ]:
evcxr_figure((4000,4000), |root| {
    let areas: Vec<_> = root.split_evenly((4, 4)).into_iter().map(|area| area.margin(40, 40, 40, 40)).collect();
    for i in 0..16 {
        let ix = 1 + (4 * i);
        render(&areas[i], bounds, &points.as_slice()[ix..(ix + 4)], &styles.as_slice()[ix..(ix + 4)], ix, 1);
    }
    Ok(())
})

What we see are the first few of the infinitely many layers of new points obtained by the closure process of $\mathbb{Z}[\zeta_4]$ under the additional degrees of freedom granted by $\mathbb{Z}[\zeta_{12}]$.

Something is happening here that at least *I* would not have expected, and I doubt that this is something you easily expect by just looking at the algebra.

Remember that the unit square is a representative "tile" of the complex plane, which would look everywhere like this if we could have started on *every* Gaussian integer.

The points we found in each round all fall onto two symmetric "rectangles" (in certain rounds they seem to almost merge into one).

This behavior is especially visible for the later rounds, with many points, but if you look closely, this is what happens for *each* round.

Now, that some symmetry shows up is not surprising - rather expected. But:
* Why do we get these rectangular shapes?
* Why do we get a pattern of **two** rectangles, not some different number?

Maybe some number theorist can explain it as a trivial consequence of some property. Or someone will point out a logic flaw in the exploration, or find a bug in the code.

In the meantime, **all I can say is that this is where math and art meet. We only are uncovering what is already there.** Nothing is being created here, only discovered and marvelled at.

### Breadth-First-Exploration from the Origin

For comparison, now let us do the naive exploration starting from the origin. It will reveal different beautiful and interesting patterns for us to look at.

In [ ]:
let (points, bounds, styles) = prepare_render::<ZZ12>(16, false, Some(2));

In [ ]:
evcxr_bitmap_figure((4000,4000), |root| {
    root.fill(&WHITE);
    let areas: Vec<_> = root.split_evenly((4, 4)).into_iter().map(|area| area.margin(40, 40, 40, 40)).collect();
    for i in 0..16 {
        render(&areas[i], bounds, &points.as_slice()[i..=i], &styles.as_slice()[i..=i], i, 1);
    }
    Ok(())
})

## Other Visualization Ideas

If you want, you can try to adapt the code to remember for each point from which direction it was discovered (i.e., which unit was added to reach it). This could reveal other interesting patterns in neighborhoods of points.

I have also been thinking about constructing a graph (remembering the edges of the connectivity), but the question is whether there is a nice way to make use of that information visually.

Another direction could be considering 3D plots with plotters and unrolling the discovery rounds into the third dimension. Plotters can also easily create animated GIFs, but those do not work in the notebook environment.

## Bonus: Trying Some Other Rings

As a bonus, let us use the code above to look at the rings $\mathbb{Z}[\zeta_{20}]$ and $\mathbb{Z}[\zeta_{24}]$. The computation takes significantly longer while the result is rather crowded and not as interesting and comprehensible as we have seen in $\mathbb{Z}[\zeta_{12}]$.

But here, the tilezz crate can show off what it can do as a generic cyclotomic ring implementation - to look at other rings, all we have to change is one type argument.

In [ ]:
let (points, bounds, styles) = prepare_render::<ZZ20>(6, true, Some(5));

In [ ]:
evcxr_bitmap_figure((3000,2000), |root| {
    root.fill(&WHITE);
    let areas: Vec<_> = root.split_evenly((2,3)).into_iter().map(|area| area.margin(40, 40, 40, 40)).collect();
    for i in 0..6 {
        render(&areas[i], bounds, &points.as_slice()[i..=i], &styles.as_slice()[i..=i], i, 1);
    }
    Ok(())
})

In [ ]:
let (points, bounds, styles) = prepare_render::<ZZ24>(6, true, Some(5));

In [ ]:
evcxr_bitmap_figure((3000,2000), |root| {
    root.fill(&WHITE);
    let areas: Vec<_> = root.split_evenly((2,3)).into_iter().map(|area| area.margin(40, 40, 40, 40)).collect();
    for i in 0..6 {
        render(&areas[i], bounds, &points.as_slice()[i..=i], &styles.as_slice()[i..=i], i, 1);
    }
    Ok(())
})

In [ ]:
let (points, bounds, styles) = prepare_render::<ZZ20>(6, false, Some(2));

In [ ]:
evcxr_bitmap_figure((3000,2000), |root| {
    root.fill(&WHITE);
    let areas: Vec<_> = root.split_evenly((2, 3)).into_iter().map(|area| area.margin(40, 40, 40, 40)).collect();
    for i in 0..6 {
        render(&areas[i], bounds, &points.as_slice()[i..=i], &styles.as_slice()[i..=i], i, 1);
    }
    Ok(())
})

In [ ]:
let (points, bounds, styles) = prepare_render::<ZZ24>(6, false, Some(2));

In [ ]:
evcxr_bitmap_figure((3000,2000), |root| {
    root.fill(&WHITE);
    let areas: Vec<_> = root.split_evenly((2, 3)).into_iter().map(|area| area.margin(40, 40, 40, 40)).collect();
    for i in 0..6 {
        render(&areas[i], bounds, &points.as_slice()[i..=i], &styles.as_slice()[i..=i], i, 1);
    }
    Ok(())
})